Restaurantes

In [1]:
import os
import csv
import re
from bs4 import BeautifulSoup

# === CONFIGURACIÓN ===
carpeta_html = "paginas tripadvisor"  # carpeta donde están los archivos HTML
salida_csv = "lugares_tripadvisor.csv"

# === VARIABLES ===
columnas = [
    "nombre", "tipo_lugar", "descripcion",
    "puntuacion", "num_opiniones",
    "ventajas", "horario", "direccion",
    "ciudad", "provincia", "pais",
    "telefono", "sitio_web", "info_relevante"
]


def limpiar_texto(txt):
    if not txt:
        return None
    txt = re.sub(r"\s+", " ", txt)
    return txt.strip(" ,;:.")


def extraer_info_tripadvisor(ruta_html):
    with open(ruta_html, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    info = {col: None for col in columnas}

    # === NOMBRE ===
    if soup.title:
        info["nombre"] = limpiar_texto(soup.title.text.split(",")[0])

    # === TIPO DE LUGAR ===
    titulo = soup.title.text.lower() if soup.title else ""
    if "restaurante" in titulo:
        info["tipo_lugar"] = "Restaurante"
    elif "hotel" in titulo or "hostal" in titulo:
        info["tipo_lugar"] = "Alojamiento"
    else:
        info["tipo_lugar"] = "Lugar turístico"

    # === DESCRIPCIÓN ===
    desc_tag = soup.find("div", {"class": re.compile("fIrGe|Wfkdv")})
    if desc_tag:
        info["descripcion"] = limpiar_texto(desc_tag.get_text(" ", strip=True))

    # === PUNTUACIÓN ===
    puntuacion_tag = soup.find(string=re.compile(r"de\s*5\s*burbujas"))
    if not puntuacion_tag:
        for tag in soup.find_all(["span", "div"]):
            aria = tag.get("aria-label")
            if aria and "de 5" in aria:
                puntuacion_tag = aria
                break
    if puntuacion_tag:
        match = re.search(r"([\d,\.]+)\s*de\s*5", str(puntuacion_tag))
        if match:
            info["puntuacion"] = match.group(1).replace(",", ".")

    # === NÚMERO DE OPINIONES ===
    num_tag = soup.find("div", {"data-automation": "bubbleReviewCount"})
    if num_tag:
        texto = num_tag.get_text(" ", strip=True)
        match = re.search(r"(\d+)", texto)
        if match:
            info["num_opiniones"] = match.group(1)
    else:
        for tag in soup.find_all(["a", "span", "div"], string=True):
            texto = tag.get_text(" ", strip=True)
            if re.search(r"\d+\s+opiniones", texto, re.IGNORECASE):
                match = re.search(r"(\d+)", texto)
                if match:
                    info["num_opiniones"] = match.group(1)
                    break

    # === VENTAJAS ===
    ventajas = []
    ventajas_titulo = soup.find("div", string=re.compile(r"VENTAJAS", re.IGNORECASE))
    if ventajas_titulo:
        contenedor = ventajas_titulo.find_next("div")
        if contenedor:
            for item in contenedor.find_all("div"):
                texto = item.get_text(" ", strip=True)
                if texto and len(texto) < 60:  # descartar bloques largos
                    ventajas.append(texto)
    if ventajas:
        info["ventajas"] = ", ".join(sorted(set(ventajas)))

    # === HORARIO ===
    horario_div = soup.find("div", {"data-automation": "hours-section"})
    if horario_div:
        texto = horario_div.get_text(" ", strip=True)
        info["horario"] = limpiar_texto(texto)
    else:
        abierto_tag = soup.find(string=re.compile(r"Abierto ahora|Horario", re.IGNORECASE))
        if abierto_tag:
            info["horario"] = limpiar_texto(abierto_tag)

    # === DIRECCIÓN ===
    maps_link = soup.find("a", href=re.compile(r"(maps\.google\.com|google\.com/maps)"))
    if maps_link:
        texto = maps_link.get_text(" ", strip=True)
        info["direccion"] = limpiar_texto(texto)

    # === CIUDAD / PROVINCIA / PAÍS ===
    if "Pontevedra" in ruta_html:
        info["ciudad"] = "Pontevedra"
        info["provincia"] = "Pontevedra"
        info["pais"] = "España"

    # === TELÉFONO ===
    for a in soup.find_all("a", href=True):
        if a["href"].startswith("tel:"):
            info["telefono"] = a["href"].replace("tel:", "")

    # === SITIO WEB ===
    for a in soup.find_all("a", href=True):
        if a["href"].startswith("http") and "tripadvisor" not in a["href"]:
            info["sitio_web"] = a["href"]

    # === INFO RELEVANTE ===
    visibles = [
        t.strip() for t in soup.stripped_strings
        if len(t.strip()) > 50
        and not any(w in t for w in ["Tripadvisor", "Facebook", "window.performance", "Foros", "Cruceros", "Hoteles"])
    ]
    if visibles:
        relevantes = []
        for t in visibles:
            if info["nombre"] and info["nombre"] in t:
                continue
            if info["direccion"] and info["direccion"] in t:
                continue
            relevantes.append(t)
        if relevantes:
            info["info_relevante"] = " | ".join(relevantes[:2])

    return info


# === PROCESAR TODOS LOS ARCHIVOS ===
os.makedirs(carpeta_html, exist_ok=True)
datos = []
for archivo in os.listdir(carpeta_html):
    if archivo.endswith(".html"):
        ruta = os.path.join(carpeta_html, archivo)
        print(f"Procesando: {archivo}")
        datos.append(extraer_info_tripadvisor(ruta))

# === GUARDAR CSV ===
with open(salida_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=columnas)
    writer.writeheader()
    writer.writerows(datos)

print(f"\n✅ CSV generado con {len(datos)} lugares: {salida_csv}")


Procesando: HOTEL GALICIA PALACE (Pontevedra)_ 2025 opiniones y precios.html
Procesando: HOTEL RIAS BAJAS (Pontevedra)_ 2025 opiniones y precios.html
Procesando: PARADOR DE PONTEVEDRA_ 2025 opiniones y precios.html

✅ CSV generado con 3 lugares: lugares_tripadvisor.csv


In [19]:
import os
import json
import re
import pandas as pd
from bs4 import BeautifulSoup

# === CONFIGURACIÓN ===
carpeta_html = r"..\carpeta_tripadvisor\paginas tripadvisor"
salida_csv = "hoteles_tripadvisor_limpio.csv"

# === FUNCIONES AUXILIARES ===
def limpiar_texto(texto):
    if not texto:
        return None
    texto = BeautifulSoup(str(texto), "html.parser").get_text(" ", strip=True)
    texto = re.sub(r"\s+", " ", texto)
    return texto.strip()

def procesar_archivo(ruta):
    with open(ruta, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    datos = {}

    # --- Nombre del hotel ---
    nombre_tag = soup.find("h1", {"id": "HEADING"})
    nombre = None
    if nombre_tag:
        nombre = nombre_tag.get_text(separator=" ", strip=True)
        nombre = re.split(r"Si es el propietario|Hacerse con el control", nombre)[0].strip()
    datos["nombre"] = nombre

    # --- Calificación general ---
    calificacion = None
    calificacion_tags = soup.find_all("div", {"data-automation": "bubbleRatingValue"})
    if calificacion_tags:
        calificacion = calificacion_tags[0].get_text(strip=True)
    datos["calificacion"] = calificacion

    # --- Número de opiniones ---
    num_opiniones = None
    opiniones_tag = soup.find("div", {"data-automation": "bubbleReviewCount"})
    if opiniones_tag:
        num_opiniones = limpiar_texto(opiniones_tag)
        # Quitar paréntesis y palabra "opiniones"
        num_opiniones = re.sub(r"[\(\)]", "", num_opiniones)
        num_opiniones = re.sub(r"opiniones?", "", num_opiniones, flags=re.IGNORECASE).strip()
    datos["num_opiniones"] = num_opiniones

    # --- Descripción ---
    desc_block = soup.find("div", {"data-automation": "aboutTabDescription"})
    descripcion = limpiar_texto(desc_block) if desc_block else None
    datos["descripcion"] = descripcion

    # --- Dirección (ubicación física, junto al ícono de ubicación) ---
    direccion = None
    direccion_botones = soup.find_all("button", class_=re.compile(r"UikNM"))
    for boton in direccion_botones:
        svg_icono = boton.find("svg")
        if svg_icono and "path" in str(svg_icono):  # asegurar que contiene el ícono
            span_texto = boton.find("span", class_=re.compile(r"AWdfh"))
            if span_texto:
                direccion = limpiar_texto(span_texto)
                break
    datos["direccion"] = direccion

    # --- Características / Servicios ---
    servicios = []
    secciones = soup.find_all("div", {"data-test-target": "amenity_text"})
    for s in secciones:
        texto = s.get_text(strip=True)
        if texto and texto.lower() not in ("servicios de la propiedad", "servicios de habitación", "tipos de habitación"):
            servicios.append(texto)

    # Agrupar los servicios por tipo si hay subtítulos
    servicios_por_tipo = {}
    bloque_servicios = soup.find_all("div", class_=re.compile("osNWb|ZTUob|lSeKz"))
    tipo_actual = None

    for elemento in bloque_servicios:
        texto = elemento.get_text(strip=True)
        if any(palabra in texto.lower() for palabra in ["servicios de la propiedad", "servicios de habitación", "tipos de habitación"]):
            tipo_actual = texto
            servicios_por_tipo[tipo_actual] = []
        elif tipo_actual and texto not in servicios_por_tipo[tipo_actual]:
            servicios_por_tipo[tipo_actual].append(texto)

    # Unir todos los servicios separados por “|”
    caracteristicas = []
    if servicios_por_tipo:
        for tipo, items in servicios_por_tipo.items():
            if items:
                caracteristicas.append(f"{tipo}: {' | '.join(items)}")
    elif servicios:
        caracteristicas.append(" | ".join(servicios))

    datos["caracteristicas"] = "; ".join(caracteristicas) if caracteristicas else None

    return datos


# === PROCESAR TODOS LOS ARCHIVOS ===
registros = []
for archivo in os.listdir(carpeta_html):
    if archivo.endswith(".html"):
        ruta = os.path.join(carpeta_html, archivo)
        print(f"Procesando: {archivo}")
        registros.append(procesar_archivo(ruta))

# === GUARDAR EN CSV ===
df = pd.DataFrame(registros)
df.to_csv(salida_csv, index=False, encoding="utf-8-sig")
print(f"\n✅ CSV generado con éxito: {salida_csv}")


Procesando: HOTEL GALICIA PALACE (Pontevedra)_ 2025 opiniones y precios.html
Procesando: HOTEL RIAS BAJAS (Pontevedra)_ 2025 opiniones y precios.html
Procesando: PARADOR DE PONTEVEDRA_ 2025 opiniones y precios.html

✅ CSV generado con éxito: hoteles_tripadvisor_limpio.csv
